# Demo CNN for Wildfire Growth Prediction

Below is starter code for a cnn solution to solve the wildfire growth challenge!

We provide infrastructure and helper functions to call and process the data.

It is up to your team to fill in necessary blanks and improve the pipeline.

In [1]:
!pip3 install torch
!pip3 install torchvision
!pip3 install tqdm
!pip3 install scikit-learn
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio as rio
import os
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import torch.optim as optim
from tqdm import tqdm
import torch.nn.functional as F

# Paths for data and fires
data_path = "./"
train_path = data_path + "train/"
test_path = data_path + "test/"
tr_fnums = ["fire1298", "fire2034", "fire2210", "fire2211", "fire2212"]
te_fnums = ["fire2214"]

In [2]:
# Util variables
device = "cpu"
target_shape = (528, 720)

# Util functions
def pad_to_fit(d, shape):
    h, w = d.shape
    pad_h = shape[0] - h
    pad_w = shape[1] - w
    if pad_h > 0 or pad_w > 0:
        pad_top = pad_h // 2
        pad_bottom = pad_h - pad_top
        pad_left = pad_w // 2
        pad_right = pad_w - pad_left

        d = np.pad(d, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='constant', constant_values=0)
    return d

def normalize(d):
    m = np.mean(d)
    s = np.std(d)
    return (d - m)/s

def tif2np(tif):
    with rio.open(tif) as src:
        data = src.read(1)  # Read the first band
    return pad_to_fit(np.nan_to_num(data, nan=0.0), target_shape)

## Functions to load data

The load fire function loads and processes data for the denoted fire. The fire is then stacked into a numpy array.

The load day function loads in a day of data for a specified fire. 

<ins>**Additional data should be loaded and specified into this function**.<ins>

In [3]:

def load_day(path, day):
    # fire_weather
    fwi = path+'/fire_weather/fire_weather_index_day{}.tif'.format(day)
    fwi = normalize(tif2np(fwi))
    # weather relative humidity
    wrh = path+'/weather/noon_relative_humidity_day{}.tif'.format(day)
    wrh = normalize(tif2np(wrh))
    # weather wind speed
    wws = path+'/weather/noon_wind_speed_day{}.tif'.format(day)
    wws = normalize(tif2np(wws))
    # Add more data here
    wwd = path+'/weather/noon_wind_direction_day{}.tif'.format(day)
    wwd = normalize(tif2np(wwd))


    tem = path+'/weather/noon_temperature_day{}.tif'.format(day)
    tem = normalize(tif2np(tem))


    return [fwi, wrh, wws, wwd, tem]

def load_fire(fire_num, split = "Train"):
    path = train_path + fire_num
    if split == "Test":
        path = test_path + fire_num
    
    ftif = path + "/fire/{}.tif".format(fire_num)
    if split == "Test":
        ftif = path + "/fire/{}_train.tif".format(fire_num)
    fdata = tif2np(ftif)

    minjd, maxjd = int(np.min(fdata[np.nonzero(fdata)])), int(np.max(fdata))
    lastjd = maxjd
    if split == "Test":
        maxjd += 21
    
    elev = normalize(tif2np(path+'/topography/dem.tif'))
    slope = normalize(tif2np(path+'/topography/slope.tif'))
    fuels = tif2np(path+'/fuels/fbp_fuels.tif')
    ignition = tif2np(path+'/fire/ignitions.tif')

    dataset = []
    gt = ignition
    cfire = ignition
    for d in range(minjd, maxjd):
        data = {}

        fuels[cfire != 0] = 0
        ft = [fuels]
        ft.extend([cfire, gt, slope, elev])
        ft.extend(load_day(path, d))
        ft = np.stack(ft)
        data['ft'] = ft

        if d < lastjd:
            gt = fdata == float(d)
            data['gt'] = gt

        cfire = np.logical_or(cfire ,gt)
        
        dataset.append(data)
    return dataset

## Create the datasets and dataloaders

<ins>Create/implement data augmentations/transformations here<ins>

In [4]:
class FireDataset(Dataset):
    def __init__(self, split="Train"):
        fnums = tr_fnums if split=="Train" else te_fnums
        self.dataset = []
        for fnum in fnums:
            self.dataset.extend(load_fire(fnum, split=split))
        print(len(self.dataset))

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]
    
trainset = FireDataset(split="Train")
trainset, valset = torch.utils.data.random_split(trainset, [0.8,0.2])
testset = FireDataset(split="Test")
trainloader = torch.utils.data.DataLoader(trainset, batch_size=6, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=6, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)

141
30


## Define the network/model

In this example, we define a simple 2 layer cnn model. 

<ins>**Modify the model as you see fit!**<ins>

In [5]:
import torch
import torch.nn as nn

class FuelEmbeddings(nn.Module):
    def __init__(self, embedding_dim):
        super(FuelEmbeddings, self).__init__()
#635, 650, 665
        unique_values = [0, 1, 2, 3, 4, 7, 13, 17, 23, 37, 101, 267, 395, 475, 635, 650, 665]
        self.unique_values = torch.tensor(unique_values).to(device)  # Unique values in the categorical feature
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(num_embeddings=len(unique_values), embedding_dim=embedding_dim)

    def forward(self, categorical_feature):
        # (B,H,W) -> (B,H,W,U) wher U is unique values count
        mask = categorical_feature.unsqueeze(-1) == self.unique_values
        matching_indices = torch.argmax(mask.int(), dim=-1)

        # Apply embedding and reshape
        # (B,H,W,U) -> (B,H,W,6) -> (B,6,H,W) in default setting
        embedded_fuel = self.embedding(matching_indices)
        embedded_reshaped_fuel = embedded_fuel.permute(0, 3, 1, 2)

        return embedded_reshaped_fuel

class CNN1(nn.Module):
    def __init__(self, embedding_dim=6, num_features=8):
        super(CNN1, self).__init__()

        self.fuelembedding = FuelEmbeddings(embedding_dim)

        # (266, 433)
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=(embedding_dim+num_features-1), out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=8),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(inplace=True)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=8),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=8, out_channels=1, kernel_size=3, stride=1, padding=1)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        categorical_feature = x[:, 0, :, :]  # Extract the categorical feature
        embedded_fuel = self.fuelembedding(categorical_feature)  # Transform the categorical feature

        # Replace the original categorical feature with the embedded feature
        x = torch.cat((embedded_fuel, x[:, 1:, :, :]), dim=1)

        x = self.conv_block1(x)
        x = self.conv_block2(x)
        out = self.sigmoid(x)

        return out
    

## Define Loss function

<ins>**Create/define/specify your own loss function here!**<ins>

In [6]:
import torch
import torch.nn as nn

class IoULoss(nn.Module):
    def __init__(self, threshold=0.5):
        super(IoULoss, self).__init__()
        self.threshold = threshold

    def forward(self, outputs, labels):
        # threshold condition is not differentiable so just use softmaxed data
        # Flatten the tensors
        outputs = outputs.view(-1)
        labels = labels.view(-1)

        # Compute the intersection
        intersection = (outputs * labels).sum()

        # Compute the union
        union = outputs.sum() + labels.sum() - intersection
        iou = intersection / (union + 1e-6)  # Add a small epsilon for numerical stability
        loss = 1 - iou
        return loss

#### Metrics for evaluation

In [7]:
from sklearn.metrics import accuracy_score, jaccard_score, f1_score

#### Train function

In [8]:
# Train
def train(model, dataloader, optimizer, criterion):
    model.train()
    running_loss = 0
    total_steps = 0
    for i, batch in enumerate(dataloader):
        ft = batch['ft'].to(device).float()
        gt = batch['gt'].to(device).float()

        optimizer.zero_grad()
        output = model(ft).squeeze()

        loss = criterion(output, gt)
        
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        total_steps += 1
    return running_loss/total_steps

#### Eval function

In [9]:
def eval(model, dataloader):
    model.eval()
    acc = []
    iou = []
    f1 = []
    total_steps = 0
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            ft = batch['ft'].to(device)
            gt = torch.flatten(batch['gt'])

            output = torch.flatten(model(ft)).squeeze().cpu()
            output = (output > 0.46)

            acc.append(accuracy_score(gt, output))
            iou.append(jaccard_score(gt, output))
            f1.append(f1_score(gt, output))
            total_steps += 1
    return sum(acc)/total_steps, sum(iou)/total_steps, sum(f1)/total_steps

#### Inference function

Saves the inference results to a submission file!

In [10]:
def inference(model, dataloader):
    model.eval()
    with torch.no_grad():
        cfire = torch.zeros(target_shape)
        for i, day in enumerate(dataloader):
            ft = day['ft'].to(device)

            # Create the submission file after 10 days
            if i > 9:
                cfire = torch.logical_or(output, cfire) # define the cumulative fire
                ft[0][1] = cfire # set the cumulative fire for the next input
                ft[0][2] = output # set the next step fire for the next input
            else:
                cfire = ft[0][1]

            output = model(ft)
            output = (output > 0.5)
    
    # Save the cumulative fire
    pred = cfire.cpu().squeeze().numpy()
    save_df = pd.DataFrame(pred)  # convert img data to df
    save_df.to_csv("./output/submission.csv", index_label='row')
    return pred

#### The training/eval/inference loop

<ins>**Define new optimizers here**<ins>

<ins>**Utilize a scheduler here**<ins>

<ins>**Change the learning rate here**<ins>

<ins>**Implement a better early stopping strategy here**<ins>

<ins>**Implement other tricks here (i.e. EMA)**<ins>


In [11]:
model = CNN1(num_features=10)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = IoULoss()
epochs = 18
best_miou = 0
for e in range(epochs):
    loss = train(model, trainloader, optimizer, criterion)
    aa, miou, mf1 = eval(model,valloader)

    if miou > best_miou:
        best_miou = miou
        cfire = inference(model, testloader)
        e = str(e)+"*"
    print(e, " avg iou loss:{:.3f} avg acc: {:.3f} avg f1: {:.3f} avg iou: {:.3f}".format(loss, aa, mf1, miou))



0*  avg iou loss:0.994 avg acc: 0.426 avg f1: 0.016 avg iou: 0.008
1*  avg iou loss:0.992 avg acc: 0.537 avg f1: 0.017 avg iou: 0.009
2*  avg iou loss:0.991 avg acc: 0.544 avg f1: 0.019 avg iou: 0.010
3*  avg iou loss:0.991 avg acc: 0.574 avg f1: 0.019 avg iou: 0.010
4*  avg iou loss:0.989 avg acc: 0.775 avg f1: 0.023 avg iou: 0.012
5*  avg iou loss:0.988 avg acc: 0.722 avg f1: 0.024 avg iou: 0.012
6*  avg iou loss:0.986 avg acc: 0.985 avg f1: 0.104 avg iou: 0.055
7  avg iou loss:0.982 avg acc: 0.743 avg f1: 0.027 avg iou: 0.014
8*  avg iou loss:0.971 avg acc: 0.989 avg f1: 0.151 avg iou: 0.082
9*  avg iou loss:0.950 avg acc: 0.989 avg f1: 0.205 avg iou: 0.114
10*  avg iou loss:0.923 avg acc: 0.995 avg f1: 0.295 avg iou: 0.174
11  avg iou loss:0.876 avg acc: 0.995 avg f1: 0.286 avg iou: 0.167
12*  avg iou loss:0.859 avg acc: 0.995 avg f1: 0.308 avg iou: 0.182
13*  avg iou loss:0.851 avg acc: 0.995 avg f1: 0.326 avg iou: 0.196
14*  avg iou loss:0.842 avg acc: 0.994 avg f1: 0.334 avg iou

## Other Ideas to implement!

<ins>**Ensemble learning - voting**<ins>

<ins>**Implement hot spot data pipeline**<ins>

<ins>**Make better use of temporal data**<ins>

<ins>**Get creative!**<ins>